In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [8]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide

# 讀取股價數據
# 設定股票代碼和日期範圍
ticker_symbol = 'AMD'
start_date = '2019-01-01'
end_date = '2019-12-31'

# 下載股票數據
stock_df = yf.download(ticker_symbol, start=start_date, end=end_date)
stock_df = stock_df.reset_index()
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df['Date'] = stock_df['Date'].dt.date
predicted_price = pd.read_csv("predicted_prices.csv", encoding='utf-8', encoding_errors = 'ignore')
predicted_price.rename(columns={predicted_price.columns[0]: 'Date'}, inplace=True)
predicted_price = predicted_price.reset_index()
predicted_price['Date'] = pd.to_datetime(predicted_price['Date'])
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
merged_df = pd.merge(stock_df, predicted_price, on='Date', how='left')
merged_df = merged_df.dropna(subset = ['predicted_mean']).reset_index(drop = True)
merged_df.drop(columns=['index'],inplace=True)

five_day_min_low = talib.MIN(merged_df['Low'], timeperiod=5) # 五天最低價當作long strategy的停損點
five_day_max_high = talib.MAX(merged_df['High'], timeperiod=5) # 五天最高價當作short strategy的停損點

merged_df['five_day_min_low'] = five_day_min_low
merged_df['five_day_max_high'] = five_day_max_high
merged_df.loc[0:3, 'five_day_min_low'] = merged_df.loc[0:3, 'Low']
merged_df.loc[0:3, 'five_day_max_high'] = merged_df.loc[0:3, 'High']
merged_df['predicted_price_up'] = (merged_df['predicted_mean'] > merged_df['predicted_mean'].shift(1)).astype(int)
merged_df['actual_price_up'] = (merged_df['Close'] > merged_df['Close'].shift(1)).astype(int)
merged_df['long_condition'] = merged_df['Close'].shift(1)<=merged_df['predicted_mean'] 
merged_df['short_condition'] = merged_df['Close'].shift(1)>merged_df['predicted_mean'] 
merged_df['long_condition'] = merged_df['long_condition'].astype(int)
merged_df['short_condition'] = merged_df['short_condition'].astype(int)
merged_df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,predicted_mean,five_day_min_low,five_day_max_high,predicted_price_up,actual_price_up,long_condition,short_condition
0,2019-02-07,22.990000,23.219999,22.320000,22.670000,22.670000,86723900,24.021129,22.320000,23.219999,0,0,0,0
1,2019-02-08,22.330000,23.280001,22.270000,23.049999,23.049999,78129300,23.965191,22.270000,23.280001,0,1,1,0
2,2019-02-11,23.049999,23.280001,22.660000,22.959999,22.959999,60578700,23.804844,22.660000,23.280001,0,0,1,0
3,2019-02-12,23.430000,23.559999,22.750000,22.820000,22.820000,67595400,23.662173,22.750000,23.559999,0,0,1,0
4,2019-02-13,22.980000,23.240000,22.709999,22.850000,22.850000,57544200,23.580233,22.270000,23.559999,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2019-12-23,44.580002,45.630001,44.389999,45.459999,45.459999,55886500,40.263574,42.150002,45.630001,1,1,0,1
222,2019-12-24,46.099998,46.610001,45.770000,46.540001,46.540001,44432200,40.585578,42.240002,46.610001,1,1,0,1
223,2019-12-26,46.990002,47.310001,45.660000,46.630001,46.630001,57562800,40.781591,42.599998,47.310001,1,1,0,1
224,2019-12-27,46.849998,46.880001,45.980000,46.180000,46.180000,36581300,41.123771,43.259998,47.310001,1,0,0,1


strategy


In [9]:
# long strategy
signals_long = np.zeros(len(merged_df))
partition_size_long = 0
stop_loss_long = 0
for i in range(len(merged_df)):
    if partition_size_long == 0:
        if merged_df['long_condition'][i] == 1:
            signals_long[i] = 1
            partition_size_long = 1
            stop_loss_long = merged_df['five_day_min_low'][i]
    if partition_size_long == 1:
        if (merged_df['Close'][i] <= stop_loss_long) or (merged_df['short_condition'][i] == 1): # 停利停損
            signals_long[i] = -1
            stop_loss_long = 0 # 停損點重置
            partition_size_long = 0 # 平倉

# short strategy
signals_short = np.zeros(len(merged_df))
partition_size_short = 0
stop_loss_short = 0
for i in range(len(merged_df)):
    if partition_size_short == 0:
        if merged_df['short_condition'][i] == 1:
            signals_short[i] = -1
            partition_size_short = -1
            stop_loss_short = merged_df['five_day_max_high'][i]
    if partition_size_short == -1:
        if (merged_df['Close'][i] >= stop_loss_short) or (merged_df['long_condition'][i] == 1):# 停利停損
            signals_short[i] = 1
            stop_loss_short = 0 # 停損點重置
            partition_size_short = 0 # 平倉

entries_long = signals_long == 1
exits_long = signals_long == -1
entries_short = signals_short == -1
exits_short = signals_short == 1
pf = vbt.Portfolio.from_signals(merged_df['Open'], 
                                    entries=entries_long,
                                    exits=exits_long, 
                                    short_entries=entries_short,
                                    short_exits=exits_short,
                                    fees = 0.000125,
                                    freq='1D',
                                    direction='both',)
# print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
print(pf.stats().to_string())
'''entries_short = signals_short == -1
exits_short = signals_short == 1
pf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')'''
# print(pf_short.stats().to_string())

Start                                                 0
End                                                 225
Period                                226 days 00:00:00
Start Value                                       100.0
End Value                                    230.563246
Total Return [%]                             130.563246
Benchmark Return [%]                         100.695954
Max Gross Exposure [%]                            100.0
Total Fees Paid                                3.968834
Max Drawdown [%]                              16.959414
Max Drawdown Duration                  53 days 00:00:00
Total Trades                                         82
Total Closed Trades                                  81
Total Open Trades                                     1
Open Trade PnL                                 4.068248
Win Rate [%]                                  61.728395
Best Trade [%]                                15.877378
Worst Trade [%]                               -7

C:\Users\Admin\AppData\Local\Temp\ipykernel_23976\1068847299.py:37: UserWarning:

direction has no effect if short_entries and short_exits are set



"entries_short = signals_short == -1\nexits_short = signals_short == 1\npf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')"

visualization


In [10]:
pf.plot().show()